In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['GR']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('glucocorticoid receptor', 4805),
 ('glutathione reductase', 1112),
 ('gc receptor', 228),
 ('glucocorticoid', 162),
 ('glyphosate resistant', 42),
 ('gemcitabine resistant', 38),
 ('glucose restriction', 37),
 ('graphene', 32),
 ('gsh reductase', 23),
 ('glycyrrhizin', 16),
 ('gefitinib resistant', 16),
 ('gingival recession', 15),
 ('growth rate', 15),
 ('gssg reductase', 15),
 ('growth restriction', 14),
 ('glucocorticosteroid receptor', 14),
 ('glycyrrhizae radix', 13),
 ('glycine arginine', 13),
 ('gp 6 and', 12),
 ('fresh and aged', 11),
 ('glutathione reductase activity', 10),
 ('glucagon receptor', 10),
 ('glycine rich', 9),
 ('gamma radiation', 9),
 ('gloeobacter rhodopsin', 8),
 ('glutamate racemase', 7),
 ('glucose reperfusion', 7),
 ('glycemic response', 7),
 ('glucorticoid receptor', 7),
 ('granulomas', 6),
 ('gastrodiae rhizoma', 6),
 ('genu recurvatum', 6),
 ('germination rate', 6),
 ('receptor gene', 6),
 ('gamma ray', 5),
 ('glucocorticoid resistant', 5),
 ('glucocor

In [6]:
try:
    disamb = load_disambiguator(shortforms[0])
    for shortform, gm in disamb.grounding_dict.items():
        for longform, grounding in gm.items():
            grounding_map[longform] = grounding
    for grounding, name in disamb.names.items():
        names[grounding] = name
    pos_labels = disamb.pos_labels
except Exception:
    pass

In [7]:
names

{'HGNC:7978': 'NR3C1',
 'GO:GO:0038023': 'signaling receptor activity',
 'CHEBI:CHEBI:24261': 'glucocorticoid',
 'CHEBI:CHEBI:36973': 'graphene',
 'CHEBI:CHEBI:15939': 'glycyrrhizinic acid',
 'MESH:D005889': 'Gingival Recession',
 'MESH:D000368': 'Aged',
 'HGNC:4192': 'GCGR',
 'MESH:D005720': 'Gamma Rays',
 'HGNC:10012': 'RHO',
 'MESH:D015424': 'Reperfusion',
 'MESH:D006099': 'Granuloma',
 'HP:HP:0002816': 'Genu recurvatum',
 'HGNC:4623': 'GSR',
 'MESH:C411470': 'glucocorticoid receptor alpha',
 'GO:GO:0043235': 'receptor complex',
 'CHEBI:CHEBI:73380': 'GR 113808',
 'MESH:D006084': 'Graft Rejection',
 'MESH:D006108': 'Graphite',
 'HGNC:17255': 'KNTC1',
 'CHEBI:CHEBI:24433': 'group',
 'CHEBI:CHEBI:79311': 'glucoraphanin'}

In [8]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-11-03 15:40:16] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.4/bio_ontology.pkl


In [9]:
result = [grounding_map, names, pos_labels]

In [10]:
result

[{'fresh and aged': 'ungrounded',
  'ga 80 recruitment': 'ungrounded',
  'gamma radiation': 'MESH:D005720',
  'gamma ray': 'MESH:D005720',
  'ganetespib resistant': 'ungrounded',
  'gastrodiae rhizoma': 'ungrounded',
  'gc receptor': 'HGNC:7978',
  'gefitinib resistant': 'ungrounded',
  'gem resistant': 'ungrounded',
  'gemcitabine resistant': 'ungrounded',
  'general regression': 'ungrounded',
  'general related': 'ungrounded',
  'genomic rescue': 'ungrounded',
  'genu recurvatum': 'HP:HP:0002816',
  'germination rate': 'ungrounded',
  'germination receptor': 'ungrounded',
  'gingival recession': 'MESH:D005889',
  'gloeobacter rhodopsin': 'ungrounded',
  'glucagon receptor': 'HGNC:4192',
  'glucocorticoid': 'HGNC:7978',
  'glucocorticoid receptor': 'HGNC:7978',
  'glucocorticoid receptor alpha': 'HGNC:7978',
  'glucocorticoid receptor complex': 'HGNC:7978',
  'glucocorticoid receptor α': 'HGNC:7978',
  'glucocorticoid resistant': 'ungrounded',
  'glucocorticosteroid receptor': 'HGNC:7

In [12]:
grounding_map, names, pos_labels = [{'fresh and aged': 'ungrounded',
  'ga 80 recruitment': 'ungrounded',
  'gamma radiation': 'MESH:D005720',
  'gamma ray': 'MESH:D005720',
  'ganetespib resistant': 'ungrounded',
  'gastrodiae rhizoma': 'ungrounded',
  'gc receptor': 'HGNC:7978',
  'gefitinib resistant': 'ungrounded',
  'gem resistant': 'ungrounded',
  'gemcitabine resistant': 'ungrounded',
  'general regression': 'ungrounded',
  'general related': 'ungrounded',
  'genomic rescue': 'ungrounded',
  'genu recurvatum': 'HP:HP:0002816',
  'germination rate': 'ungrounded',
  'germination receptor': 'ungrounded',
  'gingival recession': 'MESH:D005889',
  'gloeobacter rhodopsin': 'ungrounded',
  'glucagon receptor': 'HGNC:4192',
  'glucocorticoid': 'HGNC:7978',
  'glucocorticoid receptor': 'HGNC:7978',
  'glucocorticoid receptor alpha': 'HGNC:7978',
  'glucocorticoid receptor complex': 'HGNC:7978',
  'glucocorticoid receptor α': 'HGNC:7978',
  'glucocorticoid resistant': 'ungrounded',
  'glucocorticosteroid receptor': 'HGNC:7978',
  'glucoraphanin': 'CHEBI:CHEBI:79311',
  'glucorticoid receptor': 'HGNC:7978',
  'glucose reintroduction': 'ungrounded',
  'glucose reperfusion': 'ungrounded',
  'glucose response': 'ungrounded',
  'glucose restriction': 'ungrounded',
  'glutamate racemase': 'UP:P22634',
  'glutathione disulfide reductase': 'HGNC:4623',
  'glutathione reductase': 'HGNC:4623',
  'glutathione reductase activity': 'HGNC:4623',
  'glutathione reductase assay': 'ungrounded',
  'glycaemic response': 'ungrounded',
  'glycemic response': 'ungrounded',
  'glycine arginine': 'ungrounded',
  'glycine reductase': 'MESH:C020815',
  'glycine rich': 'ungrounded',
  'glycyrrhizae radix': 'MESH:C000707873',
  'glycyrrhizin': 'CHEBI:CHEBI:15939',
  'glyoxylate reductase': 'ungrounded',
  'glyphosate resistant': 'ungrounded',
  'good respond': 'ungrounded',
  'gp 6 and': 'HGNC:4623',
  'gpx and reductase': 'ungrounded',
  'gr': 'HGNC:4623',
  'gr113808': 'CHEBI:CHEBI:73380',
  'graft rejection': 'MESH:D006084',
  'granulomas': 'MESH:D006099',
  'graphene': 'ungrounded',
  'graphite': 'MESH:D006108',
  'green rust': 'ungrounded',
  'ground rye': 'ungrounded',
  'group': 'ungrounded',
  'growing rod': 'ungrounded',
  'growth rate': 'ungrounded',
  'growth rate inhibition': 'ungrounded',
  'growth restriction': 'ungrounded',
  'gsh reductase': 'HGNC:4623',
  'gssg reductase': 'HGNC:4623',
  'receptor gene': 'ungrounded'},
 {'MESH:D005720': 'Gamma Rays',
  'HGNC:7978': 'NR3C1',
  'HP:HP:0002816': 'Genu recurvatum',
  'MESH:D005889': 'Gingival Recession',
  'HGNC:4192': 'GCGR',
  'CHEBI:CHEBI:79311': 'glucoraphanin',
  'UP:P22634': 'murI',
  'HGNC:4623': 'GSR',
  'MESH:C020815': 'glycine reductase',
  'CHEBI:CHEBI:15939': 'glycyrrhizinic acid',
  'CHEBI:CHEBI:73380': 'GR 113808',
  'MESH:D006084': 'Graft Rejection',
  'MESH:D006099': 'Granuloma',
  'MESH:D006108': 'Graphite'},
 ['HGNC:4623', 'HGNC:7978']]

In [13]:
excluded_longforms = ['gr', 'gp6 and']

In [14]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [15]:
additional_entities = {}

In [16]:
unambiguous_agent_texts = {}

In [17]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [18]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [19]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [20]:
intersection1

[]

In [21]:
intersection2

[]

In [22]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [23]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [22]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-11-03 16:25:00] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-11-03 16:27:02] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9841605485629321 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [23]:
classifier.stats

{'label_distribution': {'HGNC:4623': 830,
  'HGNC:7978': 4024,
  'CHEBI:CHEBI:15939': 12,
  'MESH:D005720': 9,
  'HGNC:4192': 9,
  'MESH:D006099': 6,
  'ungrounded': 181,
  'MESH:C000707873': 10,
  'HP:HP:0002816': 3,
  'UP:P22634': 4,
  'MESH:C020815': 3,
  'CHEBI:CHEBI:73380': 2,
  'MESH:D005889': 13,
  'MESH:D006108': 2,
  'MESH:D006084': 2,
  'CHEBI:CHEBI:79311': 2},
 'f1': {'mean': 0.984161, 'std': 0.002167},
 'precision': {'mean': 0.975982, 'std': 0.002179},
 'recall': {'mean': 0.992584, 'std': 0.003012},
 'HGNC:4623': {'f1': {'mean': 0.971756, 'std': 0.007633},
  'pr': {'mean': 0.973494, 'std': 0.015523},
  'rc': {'mean': 0.970467, 'std': 0.017216}},
 'CHEBI:CHEBI:15939': {'f1': {'mean': 0.333333, 'std': 0.421637},
  'pr': {'mean': 0.3, 'std': 0.4},
  'rc': {'mean': 0.4, 'std': 0.489898}},
 'MESH:D005720': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'HP:HP:0002816': {'f1': {'mean': 0.2, 'std': 0.4},
  'pr': {'mean': 0

In [24]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [25]:
disamb.dump(model_name, results_path)

In [28]:
print(disamb.info())

Disambiguation model for GR

Produces the disambiguations:
	GCGR	HGNC:4192
	GR 113808	CHEBI:CHEBI:73380
	GSR*	HGNC:4623
	Gamma Rays	MESH:D005720
	Genu recurvatum	HP:HP:0002816
	Gingival Recession	MESH:D005889
	Graft Rejection	MESH:D006084
	Granuloma	MESH:D006099
	Graphite	MESH:D006108
	NR3C1*	HGNC:7978
	glucoraphanin	CHEBI:CHEBI:79311
	glycine reductase	MESH:C020815
	glycyrrhizinic acid	CHEBI:CHEBI:15939
	murI	UP:P22634

Class level metrics:
--------------------
Grounding          	Count	F1     
              NR3C1*	4024	0.98672
                GSR*	 830	0.97176
         Ungrounded	 181	0.73482
 Gingival Recession	  13	0.69333
glycyrrhizinic acid	  12	0.33333
         Ungrounded	  10	0.13333
         Gamma Rays	   9	    0.0
               GCGR	   9	0.53333
          Granuloma	   6	0.73333
               murI	   4	    0.6
    Genu recurvatum	   3	    0.2
  glycine reductase	   3	    0.6
          GR 113808	   2	    0.0
           Graphite	   2	    0.0
    Graft Rejection	   2	    0.0
  

In [29]:
model_to_s3(disamb)

In [30]:
preds = [disamb.disambiguate(text) for text in all_texts.values()]

In [31]:
texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0] == 'HGNC:10967']

In [36]:
texts[3]

'The non-neuronal monoamine transporters (OCT1, OCT2, EMT, and PMAT) play a key role in the clearance of monoamines from extracellular compartments. In a previous report we described endometrial distribution and cyclic variation of the vesicular monoamine transporter (VMAT2) mRNA and the neuronal norepinephrine transporter (NET) mRNA. In the present study we used in situ hybridization, real-time PCR and immunohistochemistry to reveal tissue distribution and cyclic variation of mRNA for the non-neuronal monoamine transporters in the human endometrium and early pregnancy decidua. We found that non-neuronal monoamine transporters are predominantly expressed in the stroma. The plasma membrane monoamine transporter (PMAT) mRNA expression peaked in the proliferative phase, whereas the extra-neuronal monoamine transporter (EMT) mRNA expression peaked in the secretory phase. The organic cation transporter 2 (OCT2) mRNA expression was exclusively detected in few scattered stromal cells and OCT1